In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rc
import tensorflow as tf
from logging_future import logger

location = "C:/Users/workspace/brad_public_workspace_on_win/SH_tongliang/data/数据库/火币日线/" 
file = "11_14_huobi_btcusdt_copy_for_ML_.csv"

In [6]:
rnn_unit = 10  # 隐藏层单元数量 ##################调整这个
input_size = 3  # 输入个数  
output_size = 1  # 输出个数
batch_size = 80  # 批量大小  #################调整这个
time_step = 20  # 时间步   
lr = 0.001  # 学习率
# 一般调整隐藏层数量、批量大小及学习率这几个超参数
# 输入和输出则由特征量和标签确定
# 本例中，以开盘价、最高价、最低价为特征量，
# 以收盘价差即涨跌作为标签

df = pd.read_csv(location + file, engine="python")
pre_data = df.iloc[:, 4].values  # 取收盘价计算标签
label = []
for i in range(1, len(pre_data)):
    label.append(round(pre_data[i] - pre_data[i-1], 4))

df.loc[1:, 'label'] = label
df['label_predict'] = df['label'].shift(-1)
data = df.iloc[:, [1, 2, 3, 5]].values  # 获取特征量及标签，类型为np.ndarray
# logger.info(data[0:5])

train_begin = 0
train_end = 206
train_times = 10  # 训练次数
test_begin = 200
test_end = 386

In [7]:
df.fillna(method='ffill', inplace = True)

In [8]:
df

,time,open,high,low,close,label,label_predict
0,2017-10-26 0:00,5712.00,5990.00,5400.00,5875.91,NaN,-136.91
1,2017-10-27 0:00,5876.00,5948.52,5669.93,5739.00,-136.91,-80.38
2,2017-10-28 0:00,5743.43,5791.80,5637.63,5658.62,-80.38,278.33
3,2017-10-29 0:00,5656.46,5977.30,5648.03,5936.95,278.33,139.95
4,2017-10-30 0:00,5940.77,6215.80,5875.00,6076.90,139.95,262.75
5,2017-10-31 0:00,6076.87,6384.91,6051.36,6339.65,262.75,223.08
6,2017-11-1 0:00,6339.18,6590.72,6285.00,6562.73,223.08,463.42
7,2017-11-2 0:00,6561.60,7100.00,6560.18,7026.15,463.42,291.41
8,2017-11-3 0:00,7026.15,7322.90,6850.01,7317.56,291.41,-12.39
9,2017-11-4 0:00,7316.88,7318.47,6967.41,7305.17,-12.39,211.77
